# Data Sources


1) Wikimedia Dump: https://meta.wikimedia.org/wiki/Data_dumps. Das sind die offiziellen XML Dumps von Wikimedia, die monatlich aktualisiert werden. Wie auf der Website angegeben, sind die Datensätze ziemlich groß – etwa 20 GB für Englisch und 8 GB für Deutsch. Die Verarbeitung wäre also ziemlich aufwendig, da man erst einmal jede Menge XML Daten bereinigen müsste. Außerdem bräuchte man zum Extrahieren des Inhalts zusätzliche Tools oder Bibliotheken. Die Dumps sind zwar die vollständigste Datenquelle, aber für eine Uni-Aufgabe wahrscheinlich zu komplex.


2) Wikimedia Dataset: https://huggingface.co/datasets/wikimedia/wikipedia. Dieser Datensatz basiert auf den Wikipedia-Dumps und enthält bereinigte Artikel in vielen Sprachen. Das wäre dann perfekt, um die Abdeckung von englischen und deutschen Artikeln miteinander zu vergleichen. Die Daten stammen aus 2023, sind also nicht ganz aktuell. Auch wenn die Artikel schon aufbereitet wurden, ist nicht ganz klar, ob weiterhin "non-entity pages" enthalten sind. Dafür ist der Datensatz einfach zu verarbeiten, weil es für jede Sprache einen eigenen Teil-Datensatz gibt.

3) Wiki40b: https://www.tensorflow.org/datasets/catalog/wiki40b. Dieser Datensatz wurde stark bereinigt. Er enthält englische und deutsche Artikel und eignet sich grundsätzlich gut für den Vergleich. Der Nachteil: Da Wiki40b ursprünglich für Machine-Learning-Zwecke gedacht ist, fehlen vermutlich einige nützliche Inhalte, was die Vollständigkeit einschränkt.


Bei meiner Recherche wollte ich einen Datensatz finden, der aktuell, vollständig und in beiden Sprachen (Englisch und Deutsch) verfügbar ist. Das Wikimedia Dataset auf Hugging Face hat sich als beste Wahl herausgestellt, weil es sich einfach verarbeiten lässt und trotzdem genug Inhalte bietet.

Beim Arbeiten mit dem Hugging-Face-Datensatz ist mir aufgefallen, dass ich anfangs gar nicht genau überlegt hatte, wie ich gezielt Artikel zum Thema „Algorithmen“ finden kann. In diesen Datensätzen gibt es keinen direkten Filter dafür, und das bloße Suchen nach dem Wort „Algorithm“ im Titel oder Text führte zu viel zu vielen unpassenden Treffern (viele Artikel hatten mit dem Thema gar nichts zu tun). Im Endeffekt waren die oben beschriebenen Datensätze nicht wirklich hilfereich und ich musste nach was anderem suchen...


Deshalb habe ich mich letztendlich für die API von **query.wikidata.org/sparql** entschieden. Die Daten stammen von Wikidata Central Database und werden in Echtzeit abgerufen. Damit könnte man sehr gut nach Entitäten, Eigenschaften, Sprachen usw. filtern und kombinierte Abfragen erstellen (was bei der späteren Analyse sehr praktisch war). Die Nutzung der API machte die Arbeit viel effizienter, da keine vollständigen Datensätze geladen, sondern nur die tatsächlich benötigten Informationen abgefragt wurden.




## Alle Algorithmen in der englischen Wikipedia. 

Diese Liste enthält keine Duplikate. Algorithmen ohne Beschreibung sind nicht enthalten.

In [ ]:
import requests

wiki_api = "https://query.wikidata.org/sparql"
query = """
SELECT DISTINCT ?algorithm ?description WHERE {
  ?class wdt:P279+ wd:Q8366.
  ?algorithm wdt:P31 ?class.
  ?algorithm schema:description ?description.
  FILTER(LANG(?description) = "en" && STRLEN(?description) > 0)
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""

headers = {
    "User-Agent": "here was my Email Address",
    "Accept": "application/sparql-results+json"
}
try:
    output = requests.get(wiki_api, params={'query': query}, headers=headers)

    data = output.json()
    
    algorithms = []

    for result in data["results"]["bindings"]:
        article = result["algorithm"]["value"]
        algorithms.append(article)

    print(len(algorithms))

except Exception as e:
    print("Error:", e) 

1453


## Alle Algorithmen in der deutschen Wikipedia. 

Diese Liste enthält keine Duplikate. Algorithmen ohne Beschreibung sind nicht enthalten.

In [ ]:
import requests

wiki_api = "https://query.wikidata.org/sparql"
query = """
SELECT DISTINCT ?algorithm ?description WHERE {
  ?class wdt:P279+ wd:Q8366.
  ?algorithm wdt:P31 ?class.
  ?algorithm schema:description ?description.

  FILTER(LANG(?description) = "de" && STRLEN(?description) > 0)
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "de". }
}
"""

headers = {
    "User-Agent": "here was my Email Address",
    "Accept": "application/sparql-results+json"
}
try:
    output = requests.get(wiki_api, params={'query': query}, headers=headers)
    
    data = output.json()
    
    algorithms = []

    for result in data["results"]["bindings"]:
        article = result["algorithm"]["value"]
        algorithms.append(article)

    print(len(algorithms))

except Exception as e:
    print("Error:", e) 

447


# Überprüfung fehlender Einträge

Zunächst war mein Ansatz, nach Einträgen zu suchen, die sowohl einen deutschen als auch einen englischen Titel besitzen. Das war aber nicht so sinnvoll, da manche deutschen Artikel denselben englischen Titel verwenden.

Auch die Suche nach einer deutschen Beschreibung war keine gute Idee, weil viele Beschreibungen leer sind oder lediglich das englische Wort wiederholen.

Danach habe ich versucht, den entsprechenden Eintrag in der jeweils anderen Wikipedia zu finden und das hat einwandfrei funktioniert.

## Einträge die in der englischen Wikipedia, aber nicht in der deutschen enthalten sind:

In [ ]:
import requests

wiki_api = "https://query.wikidata.org/sparql"
query = """
SELECT DISTINCT ?algorithm ?enArticle WHERE {
  ?class wdt:P279+ wd:Q8366.
  ?algorithm wdt:P31 ?class.

  ?algorithm schema:description ?description.
  FILTER(STRLEN(?description) > 0)

  ?enArticle schema:about ?algorithm .
  FILTER(CONTAINS(STR(?enArticle), "en.wikipedia.org"))

  FILTER NOT EXISTS {
    ?deArticle schema:about ?algorithm .
    FILTER(CONTAINS(STR(?deArticle), "de.wikipedia.org"))
  }

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""

headers = {
    "User-Agent": "here was my Email Address",
    "Accept": "application/sparql-results+json"
}
try:
    output = requests.get(wiki_api, params={'query': query}, headers=headers)

    data = output.json()
    
    algorithms = []

    for result in data["results"]["bindings"]:
        article = result["algorithm"]["value"]
        algorithms.append(article)

    print(len(algorithms))
    print(algorithms[:10])

except Exception as e:
    print("Error:", e) 

543
['http://www.wikidata.org/entity/Q2658354', 'http://www.wikidata.org/entity/Q2662711', 'http://www.wikidata.org/entity/Q2670506', 'http://www.wikidata.org/entity/Q2679145', 'http://www.wikidata.org/entity/Q2704282', 'http://www.wikidata.org/entity/Q2709102', 'http://www.wikidata.org/entity/Q2719731', 'http://www.wikidata.org/entity/Q2734005', 'http://www.wikidata.org/entity/Q2735449', 'http://www.wikidata.org/entity/Q2739975']


## Einträge die in der deutschen Wikipedia, aber nicht in der englischen enthalten sind:

In [ ]:
import requests

wiki_api = "https://query.wikidata.org/sparql"
query = """
SELECT DISTINCT ?algorithm ?deArticle WHERE {
  ?class wdt:P279+ wd:Q8366.
  ?algorithm wdt:P31 ?class.
  ?algorithm schema:description ?description.

  ?deArticle schema:about ?algorithm .
  FILTER(CONTAINS(STR(?deArticle), "de.wikipedia.org"))

  FILTER NOT EXISTS {
    ?enArticle schema:about ?algorithm .
    FILTER(CONTAINS(STR(?enArticle), "en.wikipedia.org"))
  }

  FILTER(STRLEN(?description) > 0)


  SERVICE wikibase:label { bd:serviceParam wikibase:language "de". }
}
"""

headers = {
    "User-Agent": "here was my Email Address",
    "Accept": "application/sparql-results+json"
}
try:
    output = requests.get(wiki_api, params={'query': query}, headers=headers)

    data = output.json()
    
    algorithms = []

    for result in data["results"]["bindings"]:
        article = result["algorithm"]["value"]
        algorithms.append(article)

    print(len(algorithms))
    print(algorithms[:10])

    
except Exception as e:
    print("Error:", e) 

47
['http://www.wikidata.org/entity/Q1486014', 'http://www.wikidata.org/entity/Q1530301', 'http://www.wikidata.org/entity/Q245439', 'http://www.wikidata.org/entity/Q338522', 'http://www.wikidata.org/entity/Q603110', 'http://www.wikidata.org/entity/Q965243', 'http://www.wikidata.org/entity/Q108042034', 'http://www.wikidata.org/entity/Q109418712', 'http://www.wikidata.org/entity/Q114961711', 'http://www.wikidata.org/entity/Q118904446']


# Quality & Quantity

Es gibt 543 Einträge in der englischen Wikipedia, die nicht in der deutschen vorhanden sind.
Und 47 Einträge in der deutschen Wikipedia, die nicht in der englischen zu finden sind.

Auch bei einer manuellen Überprüfung konnte ich einige der fehlenden Artikel nicht finden. Manchmal sind die entsprechenden Informationen lediglich als Unterabschnitt auf einer anderen Seite zu finden. So gibt es beispielsweise in der deutschen Wikipedia einen eigenen Artikel über Quadrupelbauer (https://de.wikipedia.org/wiki/Quadrupelbauer), während das Thema in der englischen Wikipedia nur als Unterabschnitt auf der Seite über Doubled Pawns (https://en.wikipedia.org/wiki/Doubled_pawns) behandelt wird.

Daher halte ich diesen Ansatz für eine gute Basis-Lösung.

Jedoch sehe ich ein anderes Problem mit Quality. Einige Artikel, die als „Algorithmen“ markiert sind, würde ich nicht wirklich so bezeichnen. ( z.B. Quadrupelbauer)
Das ist aber eher eine subjektive Einschätzung und hängt vom Analyseziel ab.

# Action Plan

Um Verwirrungen zu vermeiden und Coverage zu verbessern sollte die Kategorisierung von Algorithmus-Artikeln klarer definiert werden. (Wann gehört ein Artikel in die Kategorie "Algorithmus"? Was sind die Kriterien?). Danach könnte man zB (halb)automatische Tools einsetzen, um fehlende Artikel zu identifizieren.

Um Coverage zu verbessern sollten Teilabdeckungen berücksichtigt werden und nicht nur vollständig übereinstimmende Artikel. Ein Ansatz, der solche Abschnittsüberschneidungen erkennen kann, würde die Analyse genauer machen.

Einige Unterschiede entstehen vermutlich, weil Wikidata-Verknüpfungen fehlen oder unvollständig sind. In meiner Analyse habe ich z.B. die Einträge rausgefiltert, deren Beschreibung leer ist. Daher sollte man die Links zwischen der englischen und deutschen Wikipedia prüfen und ergänzen, sowie Beschreibungen hinzufügen (es macht meiner Meining keinen Sinn, wenn die Seite einfach nur da ist, aber gar keine nützlichen Infos enthält).

Um die Lücken in der Coverage zu schließen, sollte man die Übersetzung englischer Artikel ins Deutsche (und umgekehrt) fördern.

